# Obteniendo el TOTAL de vacunados y fallecidos por COVID-19 por cada semana epidemiológica de todo el Perú

En este notebook se busca obtener un dataset con el total de **fallecidos** y vacunados **(dosis 1,2 y 3)** por cada **año** y **semana epidemiológica** de todo el Perú. 

Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

## 0. Cargar librerías

In [87]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from epiweeks import Week

import functions as fn

## 1. Cargar direcciones de RawData

El dataset de vacunados **'TB_VACUNACION_COVID19.csv'** no está incluido en el repositorio de GitHub debido a su excesivo tamaño (~2gb). Sin embargo se puede descargar directamente desde la página de Datos Abiertos de COVID-19 de MINSA, [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion) y colocarlo en la dirección RawData.

In [60]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

ubigeo_url = 'RawData/TB_UBIGEOS.csv'
vaccenter_url = 'RawData/TB_EESS.csv'

## 2. Procesar el dataset de fallecidos

El dataset de fallecidos será procesado con la librería **Pandas**.

### 2.1. Leer y procesar el dataset

Como se sabe que cada fila representa a una persona, solamente se toma la columna **'fecha de fallecimiento'**.

Se añaden 3 columnas. El año, la semana epidemiológica correspondiente y un contador para contar cada caso de fallecimiento por COVID-19. Se elimina la columna **'fecha_fallecimiento'**.

In [61]:
# Leer solamente la columna fecha de fallecimiento
fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO', 'EDAD_DECLARADA']
df_fal = pd.read_csv(fal_url, sep = ';', usecols = fal_col, dtype = {'FECHA_FALLECIMIENTO':'int32',
                                                                     'EDAD_DECLARADA': 'int8'})
del fal_col

# Transformar a formato fecha
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

# Agregamos el año y semana epidemilógica de cada fallecido
fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO') 
                             
df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8'})

del df_fal['FECHA_FALLECIMIENTO']
df_fal.head()

df_fal.head()

,EDAD_DECLARADA,DEPARTAMENTO,year,epi_week
0,21,LAMBAYEQUE,2021,23
1,45,PIURA,2021,11
2,62,ICA,2021,22
3,75,AREQUIPA,2021,26
4,66,LIMA,2021,18


In [38]:
# Data type
df_fal.dtypes

EDAD_DECLARADA      int8
DEPARTAMENTO      object
year               int16
epi_week            int8
dtype: object

Se cuenta cada caso de la variable **'fallecidos'** respecto al año y semana epidemiológica. 

In [63]:
# Devuelve un dataframe con el total de FALLECIDOS por año y semana epidemiológica
epi_fal = df_fal.groupby(['year', 'epi_week']).count()
epi_fal = epi_fal.rename(columns = {'DEPARTAMENTO':'total_fallecidos'})
del epi_fal['EDAD_DECLARADA']

epi_fal.head()

total_fallecidos
year epi_week                  
2020 10                       3
     11                       3
     12                      36
     13                      64
     14                     226

Se obtiene un dataframe **'epi_fal'** con el total de fallecidos por año y semana epidemiológica de todo el Perú.

## 3. Procesar el dataset de vacunados

### 3.1. Leer y procesar el dataset

Debido al tamaño del dataset de vacunados y para preservar recursos, no se procesará la información usando **Pandas**. En su lugar se utilizará el paquete **Dask**. La variable no será un **pandas dataframe** si no un **dask dataframe** el cual está separado en n particiones de tipo **pandas dataframe**.

Se añaden 2 columnas. El año y la semana epidemiológica correspondiente para cada vacunado contra COVID-19. Se elimina la columna **'fecha_vacunacion'**.

In [64]:
# Leemos solo la columna fecha vacunación y dosis del dataset de vacunados usando dask
vac_col = ['fecha_vacunacion','dosis']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8'})
del vac_col

# Personas con 3 o menos dosis
df_vac = df_vac[df_vac['dosis'] <= 3]

# Convertimos a formato fecha la columna 'fecha_vacunacion'
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')

df_vac.head()

,dosis,year,epi_week
0,2,2021,26
1,2,2021,26
2,1,2021,45
3,2,2021,47
4,1,2021,36


### 3.3. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [7]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['dosis']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [8]:
vac_dose = crosstab4dask(df_vac)
vac_dose.head()

dosis               1      2  3
year epi_week                  
2021 6         105769      8  0
     7          92319      2  0
     8         123945     19  0
     9          35244  93107  0
     10         74408  81267  0

Se obtiene un dataset **'vac_dose'** con el número de dosis aplicada por año y semana epidemiológica de todo el Perú.

## 4. Unir los dataframes de fallecidos y dosis de vacuna por año y semana epidemiológica

In [9]:
# Unimos ambos dataframes por semana epidemiológica
epiweeks = pd.concat([epi_fal, vac_dose], axis=1)  

epiweeks = epiweeks.fillna(value = 0) # Rellenamos vacíos o Nan values con 0
del epi_fal, vac_dose, df_vac

# Cambiamos a int ya que existen Nan values y cambia a float64 automáticamente
epiweeks = epiweeks.astype('int64')

# Rename columns
epiweeks.rename(columns={1: 'total_dosis_1', 2: 'total_dosis_2', 3: 'total_dosis_3'}, inplace=True)
                                    
epiweeks

total_fallecidos  total_dosis_1  total_dosis_2  total_dosis_3
year epi_week                                                               
2020 10                       3              0              0              0
     11                       3              0              0              0
     12                      36              0              0              0
     13                      64              0              0              0
     14                     226              0              0              0
...                         ...            ...            ...            ...
2021 45                     210         840139         609340          84988
     46                     241         720438         602235          74069
     47                     249         562319         971340         252823
     48                     257         404976         790176         667473
     49                     121         163690         270282         213022

[93 rows x 4 columns]

# Obteniendo el número de muertes confirmadas y vacunados contra COVID-19 por semana epidemiológica de cada uno de los 24 departamentos del Perú 

### 2.3. Realizar una crosstab de fallecidos por COVID-19 por departamento de Perú, para cada año y semana epidemiológica

Esto se lo realiza para obtener el TOTAL de fallecidos por cada departamento y semana epidemiológica

In [40]:
falxdep = pd.crosstab(index=[df_fal['year'], df_fal['epi_week']],
                      columns=[df_fal['DEPARTAMENTO']],
                      margins = False)

falxdep = falxdep.add_suffix('_fal')
del df_fal

falxdep

DEPARTAMENTO   AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                   0           0             0             0   
     11                   0           0             0             0   
     12                   0           0             0             0   
     13                   0           4             0             0   
     14                   0           0             0            15   
...                     ...         ...           ...           ...   
2021 45                   2          10             2             7   
     46                   1          11             3             7   
     47                   1          12             3             8   
     48                   3           8             2             9   
     49                   1           4             0             4   

DEPARTAMENTO   AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                   0              0           0          0   
     11                   0              0           0          0   
     12                   1              0           0          1   
     13                   0              0           2          3   
     14                   0              2          15          2   
...                     ...            ...         ...        ...   
2021 45                   6              3           4          5   
     46                   6              2           8          4   
     47                   2              4          12          4   
     48                   8              2           9          6   
     49                   3              3           7          3   

DEPARTAMENTO   HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_fal  \
year epi_week                                 ...               
2020 10                       0            0  ...           1   
     11                       0            0  ...           1   
     12                       0            0  ...           3   
     13                       0            0  ...           6   
     14                       0            0  ...           9   
...                         ...          ...  ...         ...   
2021 45                       4            2  ...           0   
     46                       2            2  ...           1   
     47                       4            1  ...           2   
     48                       1            2  ...           0   
     49                       1            4  ...           0   

DEPARTAMENTO   MADRE DE DIOS_fal  MOQUEGUA_fal  PASCO_fal  PIURA_fal  \
year epi_week                                                          
2020 10                        0             0          0          0   
     11                        0             0          0          1   
     12                        0             0          0          1   
     13                        1             0          0          2   
     14                        0             0          0          9   
...                          ...           ...        ...        ...   
2021 45                        1             3          3         24   
     46                        2             2          1         32   
     47                        1             1          1         25   
     48                        1             1          1         38   
     49                        0             0          1         14   

DEPARTAMENTO   PUNO_fal  SAN MARTIN_fal  TACNA_fal  TUMBES_fal  UCAYALI_fal  
year epi_week                                                                
2020 10               0               2          0           0            0  
     11               0               0          0           0            0  
     12               0               0          0           0            2  
     

In [48]:
fn.variable_edad(df_fal, 'EDAD_DECLARADA')
del df_fal['DEPARTAMENTO']

df_fal

,year,epi_week,edad_cat
0,2021,23,1
1,2021,11,3
2,2021,22,5
3,2021,26,6
4,2021,18,5
...,...,...,...
201522,2020,19,7
201523,2020,37,6
201524,2020,18,6
201525,2020,32,7


In [56]:
df_fal_edad = pd.crosstab(index = [df_fal['year'], df_fal['epi_week']], columns= df_fal['edad_cat'])
df_fal_edad = df_fal_edad.rename({0: '0_17_fal', 1:'18_29_fal', 2: '30_39_fal', 3:'40_49_fal', 4: '50_59_fal',
                                  5:'60_69_fal', 6:'70_79_fal', 7:'80_more_fal'}, axis=1)
df_fal_edad

edad_cat       0_17_fal  18_29_fal  30_39_fal  40_49_fal  50_59_fal  \
year epi_week                                                         
2020 10               0          1          0          0          0   
     11               2          0          0          0          0   
     12               0          2          3          6          6   
     13               5          3          4          6          8   
     14               3          5         13         12         47   
...                 ...        ...        ...        ...        ...   
2021 45               6          6          4         18         18   
     46               6          3          9         11         34   
     47               2          4          3          9         24   
     48               2          2         10         16         27   
     49               2          0          5          8         17   

edad_cat       60_69_fal  70_79_fal  80_more_fal  
year epi_week                                     
2020 10                1          0            1  
     11                0          1            0  
     12                5          8            6  
     13               12         16           10  
     14               47         51           48  
...                  ...        ...          ...  
2021 45               31         56           71  
     46               45         62           71  
     47               41         75           91  
     48               52         48          100  
     49               20         22           47  

[93 rows x 8 columns]

Ahora tenemos el número de fallecidos por cada departamento y semana epidemiológica

## 3. Procesar el dataset de vacunados

No existe una manera directa de calcular el total de personas completamente vacunadas por departamento. Para lograrlo se prevee lo siguiente:

- El dataset de vacunados (RawData/TB_VACUNACION_COVID19.csv) solo proporciona información sobre el centro de vacunación llamado **'id_centro_vacunacion'**. NO el departamento.

- El dataset de los centros de vacunación [(RawData/TB_CENTRO_VACUNACION.csv)]() se puede utilizar para hacer **"match"** del **'id_centro_vacunacion'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica de 0 a 1894 que representa a cada distrito.

- Finalmente, con el dataset de UBIGEO [(RawData/TB_UBIGEOS.csv)]() es posible reemplazar a cada **'id_ubigeo'** con el departamento respectivo.

In [90]:
# Se lee otro dataset porque es más eficiente que separar y crear otro
vac_col = ['fecha_vacunacion','dosis', 'id_eess', 'edad']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8','id_eess':'int32', 'edad': 'int8'})
del vac_col

# Dejar solamente a personas con 2 dosis (será el dataset para el dataset en departamentos)
df_vac = df_vac[df_vac['dosis'] == 2]
del df_vac['dosis']

# Convertimos a formato fecha la columna 'fecha_vacunacion'
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')


def edad_cat(df, col):
    """Función para categorizar edad"""

    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]

    return np.select(conditions, choices, default=np.nan).astype('int8')

df_vac['edad_cat'] = df_vac.map_partitions(edad_cat, 'edad')
del df_vac['edad']

df_vac.head()

,id_eess,year,epi_week,edad_cat
0,18536,2021,26,6
1,4482,2021,26,5
3,24328,2021,47,0
13,23890,2021,43,1
16,17753,2021,46,1


In [94]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['edad_cat']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac


edad_cat       0_17_fal  18_29_fal  30_39_fal  40_49_fal  50_59_fal  \
year epi_week                                                         
2021 6                0          5          0          1          1   
     7                0          0          1          1          0   
     8                0          4          6          2          4   
     9                0      11031      33036      25031      17116   
     10               3      13327      25357      19577      13827   
     11               1      21137      34199      28493      20866   
     12               0       7547      11817       8641       5587   
     13               6      14368       9032      11398      13252   
     14              21      17026      19022      11416       7385   
     15               4      34692      16187       6695       6544   
     16               4       6069       6161       4628       2636   
     17               5      15941      18404      13108       7429   
     18               3       7586       3698       2145       1488   
     19              21       9748       8551       4540       5400   
     20               4       4343       4521       2505       2198   
     21              25       8960       5088       1713       1285   
     22              30       4188       2453       1559       1074   
     23              65       7260       4795       3560       4279   
     24              92      15491       9443       8528      22329   
     25              41      13741       7895       6746      23571   
     26              34       6266       4829       4818      43361   
     27              44      12632      17104      11193      44324   
     28              56      17789      19725      14332     162040   
     29              43      12405      14598      14110     371588   
     30              73      13213      22978     151796     564190   
     31             160      15623      27635     368534     451272   
     32             744      26216      41889     408818     169660   
     33             921      23310      67323     335741     134325   
     34             828      29018     105889     298735     114265   
     35             837      31406     101246     326109      70166   
     36            1603      17643      76304     216779      56887   
     37            2035      23381     233907     193653      56269   
     38            1369      22557     432944     182284      59601   
     39           13853     253585     845753     228902      70789   
     40           36841     800648     515918     141494      51006   
     41           11952     500726     258257      80719      33437   
     42            5837     456401     162472      59488      26999   
     43            4099     351358     123471      45704      21768   
     44            7122     232448      93413      37086      17368   
     45           84356     344709     101436      45260      20695   
     46           94880     326775     101174      45419      21307   
     47          614934     215426      75699      36436      17071   
     48          491567     173849      66676      31487      14946   
     49          122805      83411      34225      16173       7863   

edad_cat       60_69_fal  70_79_fal  80_more_fal  
year epi_week                                     
2021 6                 1          0            0  
     7                 0          0            0  
     8                 2          1            0  
     9              6815         75            3  
     10             8986        155           35  
     11            15999        480           44  
     12             5139        284           29  
     13             3733       2253        11815  
     14             5853       8752        33530  
     15             6487      10075        31162  
     16             2213       1359         6910  
     17             3241       1390         1526  
     18            

In [95]:
df_vac_edad = crosstab4dask(df_vac)
df_vac_edad = df_vac_edad.rename({0: '0_17_vac', 1:'18_29_vac', 2: '30_39_vac', 3:'40_49_vac', 4: '50_59_vac',
                                  5:'60_69_vac', 6:'70_79_vac', 7:'80_more_vac'}, axis=1)
df_vac_edad

edad_cat       0_17_vac  18_29_vac  30_39_vac  40_49_vac  50_59_vac  \
year epi_week                                                         
2021 6                0          5          0          1          1   
     7                0          0          1          1          0   
     8                0          4          6          2          4   
     9                0      11031      33036      25031      17116   
     10               3      13327      25357      19577      13827   
     11               1      21137      34199      28493      20866   
     12               0       7547      11817       8641       5587   
     13               6      14368       9032      11398      13252   
     14              21      17026      19022      11416       7385   
     15               4      34692      16187       6695       6544   
     16               4       6069       6161       4628       2636   
     17               5      15941      18404      13108       7429   
     18               3       7586       3698       2145       1488   
     19              21       9748       8551       4540       5400   
     20               4       4343       4521       2505       2198   
     21              25       8960       5088       1713       1285   
     22              30       4188       2453       1559       1074   
     23              65       7260       4795       3560       4279   
     24              92      15491       9443       8528      22329   
     25              41      13741       7895       6746      23571   
     26              34       6266       4829       4818      43361   
     27              44      12632      17104      11193      44324   
     28              56      17789      19725      14332     162040   
     29              43      12405      14598      14110     371588   
     30              73      13213      22978     151796     564190   
     31             160      15623      27635     368534     451272   
     32             744      26216      41889     408818     169660   
     33             921      23310      67323     335741     134325   
     34             828      29018     105889     298735     114265   
     35             837      31406     101246     326109      70166   
     36            1603      17643      76304     216779      56887   
     37            2035      23381     233907     193653      56269   
     38            1369      22557     432944     182284      59601   
     39           13853     253585     845753     228902      70789   
     40           36841     800648     515918     141494      51006   
     41           11952     500726     258257      80719      33437   
     42            5837     456401     162472      59488      26999   
     43            4099     351358     123471      45704      21768   
     44            7122     232448      93413      37086      17368   
     45           84356     344709     101436      45260      20695   
     46           94880     326775     101174      45419      21307   
     47          614934     215426      75699      36436      17071   
     48          491567     173849      66676      31487      14946   
     49          122805      83411      34225      16173       7863   

edad_cat       60_69_vac  70_79_vac  80_more_vac  
year epi_week                                     
2021 6                 1          0            0  
     7                 0          0            0  
     8                 2          1            0  
     9              6815         75            3  
     10             8986        155           35  
     11            15999        480           44  
     12             5139        284           29  
     13             3733       2253        11815  
     14             5853       8752        33530  
     15             6487      10075        31162  
     16             2213       1359         6910  
     17             3241       1390         1526  
     18            

### 3.3. Procesar los datasets de **UBIGEO** y **CENTROS DE VACUNACIÓN**.

#### 3.3.1. Leer y procesar los datasets

Para el dataset de UBIGEO solo se necesita las columnas 'id_ubigeo' y 'departamento'. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas 'id_centro de vacunación' e 'id_ubigeo'.

In [18]:
# Leemos los datasets de ubigeo y centros de vacunación
ubigeo = pd.read_csv(ubigeo_url, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(vaccenter_url, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

vaccenter.head()

,id_eess,departamento
0,17241,LORETO
1,14145,LORETO
2,7083,LORETO
3,549,LORETO
4,3207,LORETO


Ahora con el nuevo dataset **(vaccenter)** es posible encontrar el departamento correspondiente a 'id_centro de vacunacion' del dataset de vacunados **(df_vac)**.

#### 3.3.3. Encontrar el departamento correspondiente de todas las personas vacunadas

In [19]:
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')
del df_vac['id_eess']
del vaccenter

df_vac.head()

,year,epi_week,departamento
0,2021,26,LIMA
1,2021,26,LIMA
2,2021,47,CAJAMARCA
3,2021,43,LIMA
4,2021,46,PIURA


### 3.4. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [21]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['departamento']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [22]:
vac_dose = crosstab4dask(df_vac)
del df_vac

vac_dose = vac_dose.add_suffix('_vac')
vac_dose.head()

departamento   AMAZONAS_vac  ANCASH_vac  APURIMAC_vac  AREQUIPA_vac  \
year epi_week                                                         
2021 6                    0           0             0             0   
     7                    0           0             0             0   
     8                    0           2             0             0   
     9                  887        1975          2496          3763   
     10                 529        1083           833          4584   
     11                2726        5544          2813          6467   
     12                 352        1551           492          2038   
     13                 170         328           371           338   
     14                 175         547           328          2864   
     15                  96        4566           130         11601   
     16                 121         447           194           571   
     17                 396        3013           124           374   
     18                  21         842             9           737   
     19                1164         184            15            36   
     20                  15        6813            14           405   
     21                  33        6835          1423         11546   
     22                  72          93           721          8510   
     23                6679       33053         14382         22349   
     24                9206       15900          5634         46264   
     25                2898        3845          5409         17030   
     26                1626       25410          2022         27805   
     27                1546        6465          1280         28196   
     28                3541       18437          3641         70345   
     29                3940       12939          7313          8675   
     30                4498       26942          4685         27857   
     31                9443       40758          9814         25813   
     32               12771       39700         22122          8033   
     33                9089       23584          7837         16186   
     34                8953       14366          5078         41384   
     35                3162       13915          4117         53343   
     36                2568       29867          4999          4673   
     37                8663       24541          6768         52653   
     38                9743       12673         11791         12349   
     39                9921       83060         13913         55502   
     40               19558       97339         20635         87026   
     41               11002       38440         18342         75989   
     42               11179       32226         11599          6630   
     43                8261       15712          9861         14436   
     44                4019        8903          7550         19051   
     45                3831       14119          9002        101712   
     46                4427       17634          8753         26200   
     47               13326       49689         14422         43767   
     48                9085       41958         13279         17107   
     49                2259        7710          6045          8623   

departamento   AYACUCHO_vac  CAJAMARCA_vac  CALLAO_vac  CUSCO_vac  \
year epi_week                                                       
2021 6                    0              0           0          0   
     7                    0              0           0          1   
     8                    0              0           0          0   
     9                 2362           2726        4639       3204   
     10                2804            939        5445       3905   
     11                1424           5080        4609       3058   
     12                 776           2252        2204       1041   
     13                 242            403        9795        300   
     14                 339            535 

In [23]:
falvac_dep = pd.concat([falxdep, vac_dose], axis = 1).fillna(0)
falvac_dep

AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                   0           0             0             0   
     11                   0           0             0             0   
     12                   0           0             0             0   
     13                   0           4             0             0   
     14                   0           0             0            15   
...                     ...         ...           ...           ...   
2021 45                   2          10             2             7   
     46                   1          11             3             7   
     47                   1          12             3             8   
     48                   3           8             2             9   
     49                   1           4             0             4   

               AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                   0              0           0          0   
     11                   0              0           0          0   
     12                   1              0           0          1   
     13                   0              0           2          3   
     14                   0              2          15          2   
...                     ...            ...         ...        ...   
2021 45                   6              3           4          5   
     46                   6              2           8          4   
     47                   2              4          12          4   
     48                   8              2           9          6   
     49                   3              3           7          3   

               HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_vac  \
year epi_week                                 ...               
2020 10                       0            0  ...         0.0   
     11                       0            0  ...         0.0   
     12                       0            0  ...         0.0   
     13                       0            0  ...         0.0   
     14                       0            0  ...         0.0   
...                         ...          ...  ...         ...   
2021 45                       4            2  ...     16101.0   
     46                       2            2  ...     14096.0   
     47                       4            1  ...     26282.0   
     48                       1            2  ...     42490.0   
     49                       1            4  ...      6520.0   

               MADRE DE DIOS_vac  MOQUEGUA_vac  PASCO_vac  PIURA_vac  \
year epi_week                                                          
2020 10                      0.0           0.0        0.0        0.0   
     11                      0.0           0.0        0.0        0.0   
     12                      0.0           0.0        0.0        0.0   
     13                      0.0           0.0        0.0        0.0   
     14                      0.0           0.0        0.0        0.0   
...                          ...           ...        ...        ...   
2021 45                   1771.0        1837.0     2706.0    40989.0   
     46                   2283.0        1878.0     3011.0    35067.0   
     47                   1466.0        4326.0     3189.0    46063.0   
     48                   4074.0        3006.0     3975.0    57173.0   
     49                   1908.0        1077.0     1997.0    17197.0   

               PUNO_vac  SAN MARTIN_vac  TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                                                
2020 10             0.0             0.0        0.0         0.0          0.0  
     11             0.0             0.0        0.0         0.0          0.0  
     12             0.0             0.0        0.0         0.0          0.0  
     13             

# 6. Unir ambos dataframes

In [98]:
epiweeks_final = pd.concat([epiweeks, df_fal_edad, df_vac_edad, falvac_dep, ], axis = 1).fillna(0).astype('int64')
epiweeks_final

total_fallecidos  total_dosis_1  total_dosis_2  total_dosis_3  \
year epi_week                                                                  
2020 10                       3              0              0              0   
     11                       3              0              0              0   
     12                      36              0              0              0   
     13                      64              0              0              0   
     14                     226              0              0              0   
...                         ...            ...            ...            ...   
2021 45                     210         840139         609340          84988   
     46                     241         720438         602235          74069   
     47                     249         562319         971340         252823   
     48                     257         404976         790176         667473   
     49                     121         163690         270282         213022   

               0_17_fal  18_29_fal  30_39_fal  40_49_fal  50_59_fal  \
year epi_week                                                         
2020 10               0          1          0          0          0   
     11               2          0          0          0          0   
     12               0          2          3          6          6   
     13               5          3          4          6          8   
     14               3          5         13         12         47   
...                 ...        ...        ...        ...        ...   
2021 45               6          6          4         18         18   
     46               6          3          9         11         34   
     47               2          4          3          9         24   
     48               2          2         10         16         27   
     49               2          0          5          8         17   

               60_69_fal  ...  LORETO_vac  MADRE DE DIOS_vac  MOQUEGUA_vac  \
year epi_week             ...                                                
2020 10                1  ...           0                  0             0   
     11                0  ...           0                  0             0   
     12                5  ...           0                  0             0   
     13               12  ...           0                  0             0   
     14               47  ...           0                  0             0   
...                  ...  ...         ...                ...           ...   
2021 45               31  ...       16101               1771          1837   
     46               45  ...       14096               2283          1878   
     47               41  ...       26282               1466          4326   
     48               52  ...       42490               4074          3006   
     49               20  ...        6520               1908          1077   

               PASCO_vac  PIURA_vac  PUNO_vac  SAN MARTIN_vac  TACNA_vac  \
year epi_week                                                              
2020 10                0          0         0               0          0   
     11                0          0         0               0          0   
     12                0          0         0               0          0   
     13                0          0         0               0          0   
     14                0          0         0               0          0   
...                  ...        ...       ...             ...        ...   
2021 45             2706      40989     14199           12706       3639   
     46             3011      35067     15319           13078       3559   
     47             3189      46063     11858           33353       3451   
     48             3975      57173     10624           20124       2263   
     49             1997      17197      5198            2966        653   

               TUMBES_vac  UCAYALI_vac  
year 

## 6. Guardar el dataset

In [99]:
epiweeks_final.to_csv('Data/vacunados_y_fallecidos_x_semanaEpi.csv')

Finalmente se guarda el dataset final con el ['Total de vacunados y fallecidos por año y semana epidemiológica'](https://github.com/xxotto/covid19-peru/blob/main/Data/TOTAL_vacunados_y_fallecidos_x_semanaEpi.csv) de todo el Perú.

Disponible en el directorio [Data](https://github.com/xxotto/covid19-peru/tree/main/Data).

In [ ]:
%reset